In [ ]:
import warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import random, gc, keras

from keras import backend as K
from keras.preprocessing.image import load_img, img_to_array
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.applications.vgg16 import VGG16

%matplotlib inline

# Load Data

In [ ]:
train_df = pd.read_csv('./data/triplet/train.csv')
val_df = pd.read_csv('./data/triplet/validation.csv')
test_df = pd.read_csv('./data/triplet/test.csv')

print('Train:\t\t', train_df.shape)
print('Validation:\t', val_df.shape)
print('Test:\t\t', test_df.shape)

print('\nTrain Landmarks:\t', len(train_df['landmark_id'].unique()))
print('Validation Landmarks:\t', len(val_df['landmark_id'].unique()))
print('Test Landmarks:\t\t', len(test_df['landmark_id'].unique()))

In [ ]:
train_df.head()

# Helper Functions

In [ ]:
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})


# training set triplet generator
def train_triplet_generator(train_df, batch_size, img_size=(150, 150, 3)):
    """ training set triplet generator """
    while True:
        pass
        

# validation set triplet generator
def val_triplet_collector(size=3072, img_size=(150, 150, 3), seed=42):
    """ validation set triplet collector """
    pass

# test set reader
def test_image_collector(test_df, img_size=(150, 150, 3)):
    """ test set image and label collector """
    pass

# Define Parameters

In [ ]:
img_size = (150, 150, 3)  # target image size
margin = 0.3              # triplet loss margin
batch_size = 64           # training batch size

# Build Triplet Loss Model

In [ ]:
# Implement pre-trained VGG16 CNN model
vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=img_size)
vgg16.summary()

In [ ]:
# Define base network for triplet network
def base_net(input_shape=(150, 150, 3), trainable=False):
    """ define triplet network """
    # load pre-trained VGG16 model
    vgg16 = VGG16(include_top=False, weights='imagenet', input_shape=input_shape)
    vgg16.trainable = trainable
    
    # define sequential model
    model = Sequential(name='base_net')
    model.add(vgg16)
    model.add(Flatten(name='flatten'))
    model.add(Dense(512, activation='relu', name='fc1'))
    model.add(Dense(128, activation=None, name='fc2'))
    model.add(Lambda(lambda x: K.l2_normalize(x, axis=1), name='l2_norm'))
    
    return model

In [ ]:
base_model = base_net(input_shape=img_size, trainable=False)
base_model.summary()

In [ ]:
# Define triplet network
def triplet_net(base_model, input_shape=(150, 150, 3)):
    """ function to define triplet networks """
    # define input: anchor, positive, negative
    anchor = Input(shape=input_shape, name='anchor_input')
    positive = Input(shape=input_shape, name='positive_input')
    negative = Input(shape=input_shape, name='negative_input')
    
    # extract vector represent using CNN based model
    anchor_vec = base_model(anchor)
    pos_vec = base_model(positive)
    neg_vec = base_model(negative)
    
    # define inputs and outputs
    inputs=[anchor, positive, negative]
    outputs=[anchor_vec, pos_vec, neg_vec]
    
    # define the triplet model
    model = Model(inputs=inputs, outputs=outputs, name='triplet_net')
    
    return model

In [ ]:
triplet_model = triplet_net(base_model=base_model, input_shape=(150, 150, 3))
triplet_model.summary()

# Model Training

# Fune-tuning Model